In [1]:
#only doing vertical spreads, not single calls,puts
#no conditionals yet

import json
import datetime
import inspect
import re
from mt_classes import *
from mt_tokens import *
import unidecode
from IPython.display import display, Markdown, Latex



In [2]:
#with open("swing_1_9_22_20.json","r") as read_file:
#    data = json.load(read_file)
with open("cl_weekly_1_9_22_20.json","r") as read_file:
    data = json.load(read_file)
    
    

In [3]:

GREEN = '<div class="alert-success">'
YELLOW = '<div class="alert-warning">'
RED = '<div class="alert-danger">'
BLUE = '<div class="alert-info">'
END = '</div>'



EVENT_SELLING = 0

    
class Adj():
    pass
    
m = []

def seq_push(pos,letter,obj):
    global s,o
    
    s = s + letter
    o.append(obj)
    
    obj.index = pos
    
def filter(symb,text):
    
    text_split = text.split()
    
    for i,w in enumerate(text_split):
        
        if det_symbol(i,w):
            token = Symbol(i,w)

            return token.sym == symb
            
    return False


def has_class(tokens,cl):
    for token in tokens:
        if type(token) is cl:
            return True
    return False

def get_token(tokens,cl):
    for token in tokens:
        if type(token) is cl:
            return token
        
    return None

def get_token_after(tokens,cl1,cl2):
    
    found = False
    for token in tokens:
        if found is False:
            if inspect.isclass(cl1):
                if type(token) is cl1:
                    found = True
            else:
                if token is cl1:
                    found = True
        else:
            if type(token) is cl2:
                return token
    return None
    

count = 0


def count_shorted(tokens,timestamp,text):
    
    shorted1 = get_token(tokens,Action_shorting)
    shorted2 = get_token_after(tokens,shorted1,Action_shorting)
    
    if shorted1 and shorted2:
        print(text)
    pass

def get_stop(tokens,timestamp,text,options):

    text_split = text.split()

    symbol = get_token(tokens,Symbol)
    
    if not symbol:
        print('get stop: no symbol',text)
        return options
    
    stop = get_token(tokens,Stop)
    
    
    if stop:
        #print("get_stop: found stop at:",stop.index,len(text_split))
        stop_fields = {}
        stop_fields['ts'] = timestamp
        stop_fields['sym'] = '{}'.format(symbol)
        if len(text_split)-1 > stop.index:
            #print("get_stop more words")
            next = text_split[stop.index + 1]
            next = next.rstrip(',.')
            price = Price(0,next)
            if price.valid:
                stop_fields['type'] = 'price'
                stop_fields['price'] = price.price
                #print(symbol,'stop1:',price,next)
            else:
                if get_token_after(tokens,stop,Premium):
                    stop_fields['type'] = '2xpremium'
                else:
                    if next.lower() == 'breakeven':
                        #print('get stop breakeven')
                        stop_fields['type'] = 'breakeven'
                    else:
                        if next.lower() == 'tighter':
                            price = get_token_after(tokens,stop,Price)
                            if price:
                                stop_fields['type'] = 'tighter'
                                stop_fields['price'] = price.price
                        else:
                            if next.lower() == 'low':
                                stop_fields['type'] = 'low'
                            else:
                                if next.lower() == 'high':
                                    stop_fields['type'] = 'high'
                                else:
                                    print(symbol,'stop word:[',next,']',text)
                
        if len(stop_fields) > 2:
            if len(options):
                options[-1]['stop']= stop_fields
                
            else:
                print('here is a naket stop')
                options.append({'stop':stop_fields})

            pass # no option so have to create stop entry
    else:
        #print(symbol,"no stop")
        pass
        
        
    return options

    

def get_options(tokens,timestamp,text):
    global count
    
    options = []
    
    action = None
    symbol = None
    condition = None
    
#    print("get option entry: ",text)
    
    for i,token in enumerate(tokens):
        
        if not symbol:
            if type(token) is Symbol:
                symbol = token

        if action:
            if type(token) is Strike_date or type(token) is Strike_day:
                strike_date = token
            if type(token) is Strikes:
                strikes = token
                if type(tokens[i+1]) is Spread_type:
                    spread_type  = tokens[i+1]
                    
            #print('get_options:',symbol,strike_date,strikes)
                
            #if symbol and spread_type and strike_date and strikes:
            if symbol and strike_date and strikes:  # if no spread_type, then deduce from strikes and current price

                #if type(action) is Action_shorting:
                count = count + 1
                #print("get options, found option",count,symbol,spread_type,strike_date,strikes)
                option = {}
                option['ts'] = timestamp
                option['action'] = str(action)
                option['sym'] = '{}'.format(symbol)
                option['spread_type'] = str(spread_type)
                option['strike_date'] = '{}'.format(str(strike_date))
                option['strikes'] = strikes.aslist()
                if condition:
                    option['condition'] = True

                options.append(option)
                
                action = None

        else:
            
            if not condition:
                if type(token) is Condition:

                    condition = token;
                
                
            if type(token) is Action_shorting:
                action = token
            if type(token) is Action_shorted:
                action = token
                
#            print('action',str(action))
                
            spread_type = strike_date = strikes = None
            
    if len(options):
        pass
    else:
#        mt = GREEN + 'no options foune: ' + str(symbol) + ' ' + text + END
#        display(Markdown(mt))
        print("no options found for [{0}] text:".format(symbol),text)
    
            
    return options
            
            

def process_text(timestamp,text):
    global count
    
    
#    if filter('DKNG',text) is False:
#        return []
    
    tokens = (create_tokens(timestamp,text))
    
#    mt = YELLOW + 'tokens:' + str(tokens) + END
#    mt = '<div>' + 'tokens:' + str(tokens) + END
    #display(Markdown(mt))

#<div class="alert-warning">this is a green</div>
    print()
    
    print('text',text)
    print('tokens:',tokens)

    
    
    #get_stop(timestamp,text)
    
    options = get_options(tokens,timestamp,text)
    
    options = get_stop(tokens,timestamp,text,options)
    
    print("options: ",count,options)
#    display(Markdown(options))

#    print(options)
    
    #count_shorted(timestamp,text); return
    
#    if options is None:
#        print('process_text none',text)

    return options


    mrk = {}
    mrk['timestamp'] = timestamp
        

    
def process_element(timestamp,element):

        
    if isinstance(element,list):
        
        #remove the embedded dictionaries and concat the strings
        
        complete = ''
        
        for l in element:
            if isinstance(l,str) is not True:
                continue
                
            complete = complete + l
            
        element = complete
        
    if isinstance(element,str):
        element = unidecode.unidecode(element)  # convert the occasional unicode character like '–' to ascii '-'
        options = process_text(timestamp,element)
        return options
    
    return []

print('channel id:',data['id'])

options = []

for message in data['messages']:
#    if message['type'] != 'message':
#        continue

#    print(message)
    
    if 'text' in message.keys():
        options.extend(process_element(message['date'],message['text']))


#    try:
#        text = message['text']
#        process_element('date',text)
#    except:
#        print('failure:',message)
#        break

print()
print()
display(Markdown('<div class="alert-success">...</div>'))

for option in options:
    print()
    print("option: ",option)
    
    if "action" in option:

        if type(option['action']) is Action_shorting:
            if 'stop' in option:
                print(option['sym'],"option has stop")
            else:
                print(option['sym'],"option has no stop")
    else:
        if 'stop' in option:
                print(option['stop']['sym'],"Naked Stop")
                
with open('actions.json','w') as outfile:
    json.dump(options,outfile)


channel id: 10015910801

text 5/26: ZM - Consider shorting May (5/29) $180/185 bear call credit spread for a limit of $.45/share on a bounce.   -Gap Breakdown hourly, extended daily after retest.  Stop $176.22
tokens: [5/26:, ZM, shorting, Month[5], 2020-05-29, [180.0, 185.0], CALL, CALL, $0.45/share, Stop, Price]
options:  1 [{'ts': '2020-05-26T06:52:24', 'action': 'shorting', 'sym': 'ZM', 'spread_type': 'CALL', 'strike_date': '2020-05-29', 'strikes': [180.0, 185.0], 'stop': {'ts': '2020-05-26T06:52:24', 'sym': 'ZM', 'type': 'price', 'price': 176.22}}]

text 5/26: PYPL - Consider shorting half lot of Jun (6/5) $155/160 bear call credit spread for $.60/share, and add around $1.00 if get on a bounce.   Gap and Crap from all-time high, climactic weekly.  Stop $154.56
tokens: [5/26:, PYPL, shorting, Month[6], 2020-06-05, [155.0, 160.0], CALL, CALL, $0.6/share, Price, Stop, Price]
options:  2 [{'ts': '2020-05-26T07:33:38', 'action': 'shorting', 'sym': 'PYPL', 'spread_type': 'CALL', 'strike

<div class="alert-success">...</div>


option:  {'ts': '2020-05-26T06:52:24', 'action': 'shorting', 'sym': 'ZM', 'spread_type': 'CALL', 'strike_date': '2020-05-29', 'strikes': [180.0, 185.0], 'stop': {'ts': '2020-05-26T06:52:24', 'sym': 'ZM', 'type': 'price', 'price': 176.22}}

option:  {'ts': '2020-05-26T07:33:38', 'action': 'shorting', 'sym': 'PYPL', 'spread_type': 'CALL', 'strike_date': '2020-06-05', 'strikes': [155.0, 160.0], 'stop': {'ts': '2020-05-26T07:33:38', 'sym': 'PYPL', 'type': 'price', 'price': 154.56}}

option:  {'ts': '2020-05-26T07:41:19', 'action': 'shorting', 'sym': 'HES', 'spread_type': 'PUT', 'strike_date': '2020-05-29', 'strikes': [46.0, 44.0], 'stop': {'ts': '2020-05-26T07:41:19', 'sym': 'HES', 'type': 'price', 'price': 46.25}}

option:  {'ts': '2020-05-26T07:58:17', 'action': 'shorting', 'sym': 'AAPL', 'spread_type': 'PUT', 'strike_date': '2020-05-29', 'strikes': [315.0, 310.0], 'condition': True, 'stop': {'ts': '2020-05-26T07:58:17', 'sym': 'AAPL', 'type': 'price', 'price': 315.34}}

option:  {'ts':

<div class="alert-warning">this is a green</div>

<div class="alert-danger">this is a green</div>

foo <font color='red'>bar</font> foo

In [ ]:
print(options[5])

In [ ]:
if 'action' in options[5]:
    print('abc')

In [ ]:
{'timestamp': '2020-05-26T06:52:24', 'action': 'shorting', 'sym': 'ZM', 'spread_type': 'CALL', 'strike_date': '2020-05-29', 'strikes': [180.0, 185.0], 'stop': {'timestamp': '2020-05-26T06:52:24', 'sym': 'ZM', 'type': 'price', 'price': 176.22}}

In [ ]:
x = {"ages":[30,3]}

In [ ]:
x


In [ ]:
json.dumps(x)

In [ ]:
Strikes(0,'$180/185')